## Face Detection 
- Verifies that the image consisting a face or not?
- Face Detection task can be broken down into two steps
    * Classification task - whether there are any face present in the image
    * Localization task - finding the location of the face in the image(x,y,width,height)


### Factors that affect the Facedetection accuracy
- One of the most important parts of face detection is the algorithm used for detecting the facial features.
- The quality of the input image or video. Poor lighting, low resolution, and occlusions can make it difficult for the face detection algorithm to accurately detect facial features.
- The size and orientation of the face can also affect the accuracy of face detection. If the face is too small or too large in the image or if the face is tilted or rotated,it can be difficult for the algorithm to detect the facial features. 


### Improvements
- One common technique is to use machine learning algorithms, such as convolutional neural networks (CNNs), to improve the accuracy of facial feature detection. Additionally, techniques such as image pre-processing, data augmentation, and ensemble learning can also be used to improve the accuracy of face detection.
- Another important technique for improving face detection is to use multiple algorithms in combination, such as a combination of Haar cascades and deep learning models
- Finally, it is important to continuously evaluate and test the performance of the face detection system under different conditions to identify areas for improvement and refine the algorithm and system design.

## Using Viola-Jones Classifier
- In OpenCV, we have several trained Haar Cascade models which are saved as XML files. Instead of creating and training the model from scratch, we use this file. 
- We are going to use “haarcascade_frontalface_alt2.xml” file in this project
- works on grayscale images

#### 1. find the path to the “haarcascade_frontalface_alt2.xml” and “haarcascade_eye_tree_eyeglasses.xml” files.

In [1]:
import cv2
import os

In [2]:
cv2.__file__

'C:\\Users\\DELL\\anaconda3\\envs\\CV\\lib\\site-packages\\cv2\\__init__.py'

In [3]:
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

#### 2.  load the classifier
- We are using two classifiers, one for detecting the face and others for detection eyes

In [4]:
faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

In [5]:
path = r"C:\Users\DELL\Jupyter Notebook\Images\Group photos"

In [6]:
img = cv2.imread(path+"\grey-student.jpg")
img.shape

(720, 881, 3)

In [7]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray.shape

(720, 881)

In [8]:
# cv2.namedWindow('Original',cv2.WINDOW_NORMAL)
# cv2.resizeWindow('Original', 600,400)
# cv2.imshow('Original', img) 

# cv2.namedWindow('gray',cv2.WINDOW_NORMAL)
# cv2.resizeWindow('gray', 600,400)
# cv2.imshow('gray', gray) 

# cv2.waitKey(0) 
# cv2.destroyAllWindows()

In [37]:
faces = faceCascade.detectMultiScale(gray,scaleFactor=1.3,
                                        minNeighbors=5)

# Draw rectangles around the faces and eyes
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
    
cv2.namedWindow('Original',cv2.WINDOW_NORMAL)
cv2.resizeWindow('Original', 600,400)
cv2.imshow('Original', img) 

cv2.waitKey(0) 
cv2.destroyAllWindows()

In [38]:
def detect_face_img(img_name):
    path = r"C:\Users\DELL\Jupyter Notebook\Images\Group photos\\"
    
    src = path+img_name
    
    cascPathface = os.path.dirname(
        cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
    cascPatheyes = os.path.dirname(
        cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"
    
    img = cv2.imread(src)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(gray,scaleFactor=1.3,
                                        minNeighbors=5)

    # Draw rectangles around the faces and eyes
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)


    cv2.namedWindow('Original',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Original', 600,400)
    cv2.imshow('Original', img) 

    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [12]:
detect_face_img("grey-student.jpg")

In [39]:
detect_face_img("gvt-child.jpg")

In [40]:
detect_face_img("leavers.jpg")

In [41]:
detect_face_img("leavers-group-photo.jpg")

In [42]:
detect_face_img("school-twenty-seven-april_d.webp")

### Not detecting 
- africans faces
- face with sunglasses
- face with hair covers forehead
- 

#### 3. open the webcam using this simple OpenCV one-liner code
- The read() function returns:
- The actual video frame read (one frame on each loop)
- A return code
- The return code tells us if we have run out of frames, which will happen if we are reading from a file. This doesn’t matter when reading from the webcam since we can record forever, so we will ignore it.


In [43]:
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    frame = cv2.flip(frame,1)
    #For this specific classifier to work, we need to convert the frame into greyscale.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #to run the classifier cascade over the image
    faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,
                                        minNeighbors=5,
                                        minSize=(60,60),
                                        flags=cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h),(0,255,0), 2)
        faceROI = frame[y:y+h,x:x+w]
        eyes = eyeCascade.detectMultiScale(faceROI)
        for (x2, y2, w2, h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)
    
    #The variable faces now contain all the detections for the target image.
    #To show the detected face, we will draw a rectangle over it using face cordinates
    cv2.imshow("Face Detection",frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
    
video_capture.release()
cv2.destroyAllWindows()

## Face Recognition
- we are going to describe face recognition using deep learning.
- We make use of face embedding in which each face is converted into a vector and this technique is called deep metric learning.


#### 1. Face Detection
- The very first task we perform is detecting faces in the image or video stream. Now that we know the exact location/coordinates of face, we extract this face for further processing ahead.

#### 2. Feature Extraction
-  Now that we have cropped the face out of the image, we extract features from it.
- A neural network takes an image of the person’s face as input and outputs a vector which represents the most important features of a face. In machine learning, this vector is called embedding and thus we call this vector as face embedding.
- While training the neural network, the network learns to output similar vectors for faces that look similar
- Now after training the network, the network learns to output vectors that are closer to each other(similar) for faces of the same pers in a fileo
- We recognise the face if the generated embedding is closer or similar to any other embeddingn(looking similar)
- We will use a pre-trained network trained by Davis King on a dataset of ~3 million images. The network outputs a vector of 128 numbers which represent the most important features of a face.
- We pass all the images in our data to this pre-trained network to get the respective embeddings and save these embeddings in a file for the next step.

#### 3. Comparing Faces
- here we recognise a new image that is not in our data.
-  compute the face embedding for the image using the same network we used above and then compare this embedding with the rest of the embeddings we have in a file.
- We recognise the face if the generated embedding is closer or similar to any other embedding

### Libraries
- The dlib library, maintained by Davis King, contains our implementation of “deep metric learning” which is used to construct our face embeddings used for the actual recognition process.
- The face_recognition  library, created by Adam Geitgey, wraps around dlib’s facial recognition functionality, and this library is super easy to work with and we will be using this in our code. Remember to install dlib library first before you install face_recognition.

### Extracting features from Face

In [16]:
!pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25854 sha256=7b252d0d82c98457d439a73136cdffefa5b2b9ded0d924881d31486a2bec075d
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\e2\73\ca\f8ea71e39a18de34c287a665e8e821f19816dfe98996118a25
Successfully built imutils


In [14]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os

In [16]:
#get paths of each file in folder named Images
#Images here contains my data(folders of various persons)
imagePaths = list(paths.list_images('Images'))
knownEncodings = []
knownNames = []

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    name = imagePath.split(os.path.sep)[-2]
    # load the input image and convert it from BGR (OpenCV ordering)
    # to dlib ordering (RGB)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #Use Face_recognition to locate faces
    boxes = face_recognition.face_locations(rgb,model='hog')
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)
    # loop over the encodings
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)

#save emcodings along with their names in dictionary data
data = {"encodings": knownEncodings, "names": knownNames}
#use pickle to save data into a file for later use
f = open("face_enc", "wb")
f.write(pickle.dumps(data))
f.close()

In [44]:
def get_encoding(img):
    img = face_recognition.load_image_file(img)
#     rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = face_recognition.face_locations(img,model='hog')
    enc = face_recognition.face_encodings(rgb,boxes)
    return enc

In [45]:
e  = get_encoding(r"C:\Users\DELL\Jupyter Notebook\Images\Dipendra\dip5.jpg")

In [46]:
import json
import numpy as np

In [47]:
e

[array([-0.04244663,  0.088769  ,  0.05407013, -0.05314859, -0.09558126,
        -0.03008778, -0.05468063, -0.08068252,  0.14236121, -0.06268652,
         0.24435277, -0.01577381, -0.27382144,  0.02249621, -0.02079305,
         0.14156501, -0.13798869, -0.05442272, -0.16349526, -0.10314798,
        -0.00265061,  0.06218552,  0.02002058, -0.02206636, -0.06264982,
        -0.26298031, -0.07937493, -0.09183647,  0.0860993 , -0.12760018,
         0.03296038,  0.03192517, -0.1331813 ,  0.0072698 ,  0.03899386,
         0.01524134, -0.02672438, -0.10770703,  0.13513732, -0.05833907,
        -0.18521146, -0.02475039,  0.02809101,  0.20679039,  0.19525495,
         0.01432962,  0.00716137, -0.12094369,  0.06193394, -0.27590922,
         0.03955603,  0.12927254, -0.02298553,  0.09455177,  0.08733436,
        -0.18320122,  0.02923244,  0.12889469, -0.12786748,  0.02048741,
         0.10552447, -0.06819022, -0.059429  , -0.11837339,  0.19719313,
         0.09564072, -0.11827008, -0.15199099,  0.1

In [32]:
j_s = json.dumps(e[0].tolist())


In [33]:
np.array(json.loads(j_s))

array([-3.88793461e-02,  4.80120778e-02,  3.05329375e-02,  2.76116543e-02,
       -1.38187766e-01, -6.79458380e-02,  3.84580344e-05, -8.84625018e-02,
        8.13399181e-02, -6.20967522e-02,  1.74074993e-01, -5.71883023e-02,
       -2.17736661e-01, -5.09011596e-02,  2.73617711e-02,  1.02648467e-01,
       -1.03422657e-01, -1.42723471e-02, -8.51526260e-02, -1.05434969e-01,
        2.66750902e-02,  7.28340447e-02,  3.98029275e-02,  8.87039304e-03,
       -7.88770691e-02, -2.52398938e-01, -9.60173532e-02, -6.58060387e-02,
        8.21518153e-02, -5.44406027e-02,  1.30003262e-02,  5.66741005e-02,
       -2.65963942e-01, -4.23976146e-02, -2.23812349e-02,  8.51448923e-02,
       -5.03989831e-02, -7.46729895e-02,  1.94131389e-01,  2.60139536e-03,
       -1.50824413e-01,  3.74736786e-02,  5.49237542e-02,  1.60275787e-01,
        1.76750898e-01, -4.28585187e-02, -1.62193589e-02, -2.45095138e-02,
        6.21440075e-02, -1.89621300e-01,  2.35786103e-02,  1.66055515e-01,
        1.35678738e-01,  

In [48]:
data["encodings"][0].shape

(128,)

### Face Recognition in Live webcam Feed

In [35]:
import imutils
import time

In [49]:
#find path of xml file containing haarcascade file 
cascPathface = os.path.dirname(
 cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier(cascPathface)
# load the known faces and embeddings saved in last file
data = pickle.loads(open('face_enc', "rb").read())
 
print("Streaming started")
video_capture = cv2.VideoCapture(0)

# loop over frames from the video file stream
while True:
    # grab the frame from the threaded video stream
    ret, frame = video_capture.read()
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
 
    # convert the input frame from BGR to RGB 
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # the facial embeddings for face in input
    encodings = face_recognition.face_encodings(rgb)
    
    names = []
    # loop over the facial embeddings incase
    # we have multiple embeddings for multiple fcaes
    for encoding in encodings:
       #Compare encodings with encodings in data["encodings"]
       #Matches contain array with boolean values and True for the embeddings it matches closely
       #and False for rest
        matches = face_recognition.compare_faces(data["encodings"],
         encoding)

        #set name =inknown if no encoding matches
        name = "Unknown"
        # check to see if we have found a match
        if True in matches:
            #Find positions at which we get True and store them
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            # loop over the matched indexes and maintain a count for
            # each recognized face
            for i in matchedIdxs:
                #Check the names at respective indexes we stored in matchedIdxs
                name = data["names"][i]
                #increase count for the name we got
                counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)
            
         # update the list of names
        names.append(name)
        # loop over the recognized faces
        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            # draw the predicted face name on the image
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
             0.75, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

Streaming started


In [26]:
if not []:
    print("Empty")
    
else:
    print("Not Empty!")

Empty
